In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAKVLfwEAAAAA9U65FTL2ZrQgx1u1kgA94uCDzt8%3D9g59OKwN90CZ3BeWlqsMQfoFkLzUWhk7oYfz5917X90ygMxium'

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [10]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "@hm (dress OR dresses OR shirt OR shirts OR pants OR skirt OR skirts OR clothes) lang:en -is:retweet"
start_time = "2022-08-13T00:00:00.000Z"
end_time = "2022-08-19T00:00:00.000Z"
max_results = 100

In [11]:
url = create_url(keyword, start_time,end_time, max_results)

In [12]:
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [13]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "961713792005570566",
            "conversation_id": "1559186467494182914",
            "created_at": "2022-08-18T18:16:05.000Z",
            "id": "1560329702027444225",
            "in_reply_to_user_id": "184564984",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "referenced_tweets": [
                {
                    "id": "1559186467494182914",
                    "type": "replied_to"
                }
            ],
            "reply_settings": "everyone",
            "source": "Twitter for iPhone",
            "text": "@NooPuddles @hm have fabulous dresses that go well with leggings and they never need ironing \ud83d\ude0a"
        },
        {
            "author_id": "3908996909",
            "conversation_id": "1560320303657619456",
            "crea

In [14]:
json_response.keys()

dict_keys(['data', 'includes', 'meta'])

In [15]:
hm_df = pd.DataFrame(json_response['data'])

In [16]:
hm_df

,author_id,source,conversation_id,public_metrics,in_reply_to_user_id,text,reply_settings,referenced_tweets,lang,created_at,id,geo
0,961713792005570566,Twitter for iPhone,1559186467494182914,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",184564984,@NooPuddles @hm have fabulous dresses that go ...,everyone,"[{'type': 'replied_to', 'id': '155918646749418...",en,2022-08-18T18:16:05.000Z,1560329702027444225,NaN
1,3908996909,Twitter for iPhone,1560320303657619456,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,H&amp;M is Being Sued For “Misleading” Sustain...,everyone,NaN,en,2022-08-18T17:38:44.000Z,1560320303657619456,NaN
2,1458792594339598337,Typefully,1560314352380481536,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1458792594339598337,@Nestle @hm\n\n80% of discarded clothes are in...,everyone,"[{'type': 'replied_to', 'id': '156031435624343...",en,2022-08-18T17:15:07.000Z,1560314358848032769,NaN
3,1549124854812639232,Twitter for Android,1560264391802572800,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,#outfitoftheday @hm pants @VictoriasSecret cro...,everyone,NaN,en,2022-08-18T13:56:33.000Z,1560264391802572800,NaN
4,2837589067,Twitter for Android,1560238655033053184,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1403415957662437376,@climatecorporat @ZARA @UniqloUSA @hm Anything...,everyone,"[{'type': 'replied_to', 'id': '156024155013919...",en,2022-08-18T13:53:46.000Z,1560263688048689152,NaN
5,1649810408,Sendible,1560254729837576192,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,Neutral tones with a pop of colour for the win...,everyone,NaN,en,2022-08-18T13:18:10.000Z,1560254729837576192,NaN
6,258837672,Instagram,1560079584304971777,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,"8/17/22 LOTD Pants from @meijerstores , lace t...",everyone,NaN,en,2022-08-18T01:42:12.000Z,1560079584304971777,NaN
7,176447378,Instagram,1559964386340376576,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,"Not flattering, I know, but wearing an oversiz...",everyone,NaN,en,2022-08-17T18:04:27.000Z,1559964386340376576,NaN
8,967306742148329473,Twitter for Android,1559938640054308866,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1513986674920984576,@Cj_Kirin2 @hm I use to struggle with finding ...,everyone,"[{'type': 'replied_to', 'id': '155995347875096...",en,2022-08-17T17:23:29.000Z,1559954079098494976,NaN
9,220536453,Twitter for Android,1559947359664095233,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,Today's fashion choice✌🏽✌🏽✌🏽\nOn a budget😉\n@h...,everyone,NaN,en,2022-08-17T16:56:47.000Z,1559947359664095233,NaN


In [17]:
#next request for zara

In [18]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "@zara (dress OR dresses OR shirt OR shirts OR pants OR skirt OR skirts OR clothes) lang:en -is:retweet"
start_time = "2022-08-13T00:00:00.000Z"
end_time = "2022-08-19T00:00:00.000Z"
max_results = 100

In [19]:
url = create_url(keyword, start_time,end_time, max_results)

In [20]:
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [21]:
json_response['data']

[{'id': '1560364876853280770',
  'created_at': '2022-08-18T20:35:51.000Z',
  'text': '@EJGSH @ZARA I tried, literally nobody gives a shit. I’m so pissed $40 pants and they’re just like “sorry buy a new pair”',
  'lang': 'en',
  'conversation_id': '1560344787659661312',
  'reply_settings': 'everyone',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1560354922603872257'}],
  'in_reply_to_user_id': '30011210',
  'source': 'Twitter for iPhone',
  'author_id': '23023560',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 1,
   'like_count': 1,
   'quote_count': 0}},
 {'id': '1560349998528724995',
  'created_at': '2022-08-18T19:36:44.000Z',
  'text': 'Are @ZARA kids clothes true to size? or a little smaller?',
  'lang': 'en',
  'conversation_id': '1560349998528724995',
  'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'author_id': '1162452050896203776',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 1,
   'like_count': 0,
   'quote_count': 0}},
 {

In [22]:
zara_df = pd.DataFrame(json_response['data'])

In [23]:
zara_df

,id,created_at,text,lang,conversation_id,reply_settings,referenced_tweets,in_reply_to_user_id,source,author_id,public_metrics,geo
0,1560364876853280770,2022-08-18T20:35:51.000Z,"@EJGSH @ZARA I tried, literally nobody gives a...",en,1560344787659661312,everyone,"[{'type': 'replied_to', 'id': '156035492260387...",30011210,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
1,1560349998528724995,2022-08-18T19:36:44.000Z,Are @ZARA kids clothes true to size? or a litt...,en,1560349998528724995,everyone,NaN,NaN,Twitter for iPhone,1162452050896203776,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
2,1560344787659661312,2022-08-18T19:16:01.000Z,Bought a pair of pants from @ZARA had them for...,en,1560344787659661312,everyone,NaN,NaN,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN
3,1560263688048689152,2022-08-18T13:53:46.000Z,@climatecorporat @ZARA @UniqloUSA @hm Anything...,en,1560238655033053184,everyone,"[{'type': 'replied_to', 'id': '156024155013919...",1403415957662437376,Twitter for Android,2837589067,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
4,1560035225190776846,2022-08-17T22:45:56.000Z,Disappointing service from @ZARA. \nOrdered a...,en,1560035225190776846,mentionedUsers,NaN,NaN,Twitter for Android,1433752245242408964,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
5,1560014765300371457,2022-08-17T21:24:38.000Z,@ZARA I bought a dark khaki bodysuit at Zara. ...,en,1560014765300371457,mentionedUsers,NaN,346742249,Twitter for iPhone,867889480031756288,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
6,1559965489400528896,2022-08-17T18:08:50.000Z,@UnderRynovation @ZARA Girl they gave me the w...,en,1556650725157081089,everyone,"[{'type': 'replied_to', 'id': '155666021001764...",173473168,Twitter for iPhone,158125606,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
7,1559782824659542016,2022-08-17T06:02:59.000Z,"@ZARA Zara, I'm a huge fan of your clothes, as...",en,1557713473580068864,everyone,"[{'type': 'replied_to', 'id': '155771347358006...",346742249,Twitter for Android,1444055804492013575,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
8,1559709553830137856,2022-08-17T01:11:50.000Z,@CaroleGillis @Lammles @Wrangler Ooof I had a ...,en,1559696919751888896,everyone,"[{'type': 'replied_to', 'id': '155969691975188...",41591669,Twitter for iPhone,1666841995,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
9,1559669056684294145,2022-08-16T22:30:55.000Z,@zara fuck you for not letting buy my skirt,en,1559669056684294145,everyone,NaN,346742249,Twitter for iPhone,1456093411228987395,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN


In [24]:
#combining dataframes

In [25]:
df = zara_df.append(hm_df,ignore_index=True,sort=False)


In [26]:
df

,id,created_at,text,lang,conversation_id,reply_settings,referenced_tweets,in_reply_to_user_id,source,author_id,public_metrics,geo
0,1560364876853280770,2022-08-18T20:35:51.000Z,"@EJGSH @ZARA I tried, literally nobody gives a...",en,1560344787659661312,everyone,"[{'type': 'replied_to', 'id': '156035492260387...",30011210,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
1,1560349998528724995,2022-08-18T19:36:44.000Z,Are @ZARA kids clothes true to size? or a litt...,en,1560349998528724995,everyone,NaN,NaN,Twitter for iPhone,1162452050896203776,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
2,1560344787659661312,2022-08-18T19:16:01.000Z,Bought a pair of pants from @ZARA had them for...,en,1560344787659661312,everyone,NaN,NaN,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN
3,1560263688048689152,2022-08-18T13:53:46.000Z,@climatecorporat @ZARA @UniqloUSA @hm Anything...,en,1560238655033053184,everyone,"[{'type': 'replied_to', 'id': '156024155013919...",1403415957662437376,Twitter for Android,2837589067,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
4,1560035225190776846,2022-08-17T22:45:56.000Z,Disappointing service from @ZARA. \nOrdered a...,en,1560035225190776846,mentionedUsers,NaN,NaN,Twitter for Android,1433752245242408964,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
74,1558452009459748868,2022-08-13T13:54:48.000Z,@greenwashdotcom @_DirtyFashion_ @qz @hm @Higg...,en,1558435691989975041,everyone,"[{'type': 'replied_to', 'id': '155843569198997...",1491800642926514178,Twitter for iPhone,1224824519996723206,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
75,1558364150518239232,2022-08-13T08:05:41.000Z,@avneetkaur_13 I wore this outfit yesterday bu...,en,1558364150518239232,everyone,NaN,788062532682420224,Instagram,1437114287894327298,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'place_id': '7929cea6bd5b32bd', 'coordinates'..."
76,1558359664215179264,2022-08-13T07:47:51.000Z,#sjcartierltd #reselling #brandnew &amp; #used...,en,1558359664215179264,everyone,NaN,NaN,Twitter for Android,1273298286204661760,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
77,1558302374686588928,2022-08-13T04:00:12.000Z,i liked my outfit yesterday 🙆🏻‍♀️\npolo: @uniq...,en,1558302374686588928,everyone,NaN,NaN,Instagram,95186322,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN


In [27]:
df.to_csv('../data/final_tweet_dataset.csv', index=False)